# Cloud segmentation in TF2.Keras

In [77]:
import pandas as pd
import tensorflow as tf
import math
import os
import cv2

In [78]:
DATA_DIR = '/workspace/data/clouds'
TRAIN_CSV_DIR = DATA_DIR + '/train.csv'
TRAIN_IMAGE_DIR = DATA_DIR + '/train_images'
TEST_IMAGE_DIR = DATA_DIR + '/test_images'

In [79]:
BATCH_SIZE=32
N_EPOCHS=100
INPUT_SHAPE=(1400, 2100)
TRAIN_SIZE=(480, 640)
# TODO: Labels array and make rle arrays from this. Then implement loss. 
LABELS=('Fish', 'Flower', 'Sugar', 'Gravel')
N_LABELS = len(LABELS)

## Preprocessing

In [80]:
def load_data(csv_filename, image_dir):
    # Load csv using pandas 
    df = pd.read_csv(csv_filename)

    # Remove null values
    df = df[~df['EncodedPixels'].isna()]

    # Split Image and labels
    df['Image'] = df['Image_Label'].apply(lambda x: x.split('_')[0])
    df['Label'] = df['Image_Label'].apply(lambda x: x.split('_')[1])

    # Group by image
    df = df.groupby('Image')['Label', 'EncodedPixels'].agg(tuple).reset_index()

    return df

train_df = load_data(TRAIN_CSV_DIR, TRAIN_IMAGE_DIR)
# train_df.head()

In [81]:
train_df.head()

,Image,Label,EncodedPixels
0,0011165.jpg,"(Fish, Flower)",(264918 937 266318 937 267718 937 269118 937 2...
1,002be4f.jpg,"(Fish, Flower, Sugar)",(233813 878 235213 878 236613 878 238010 881 2...
2,0031ae9.jpg,"(Fish, Flower, Sugar)",(3510 690 4910 690 6310 690 7710 690 9110 690 ...
3,0035239.jpg,"(Flower, Gravel)",(100812 462 102212 462 103612 462 105012 462 1...
4,003994e.jpg,"(Fish, Gravel, Sugar)",(2367966 18 2367985 2 2367993 8 2368002 62 236...


## Utility Functions

In [109]:
def resize_image(img, req_size):
    assert len(req_size) == 2
    if img.shape[:2] != req_size:
        img = cv2.resize(img, req_size[::-1])
    return img

def rle2mask(rle, input_shape):
    'Convert Run length encoding to mask.'
    height, width = input_shape
    mask = np.zeros(height*width).astype(np.uint8)
    array = np.array([int(x) for x in rle.split(' ')])
    begins, lengths = array[0::2], array[1::2]
    for i, begin in enumerate(begins):
        mask[int(begin):int(begin+lengths[i])] = 1
    return mask.reshape(height, width)

def rles2arr(label_set, rle_set, input_shape, req_shape, n_classes):
    'Convert rle dictionary to multi-dimensional array.'
    assert len(label_set) == len(rle_set)
    mask_arr_shape = (*req_shape, n_classes)
    mask_arr = np.zeros(mask_arr_shape).astype(np.uint8)
    
    for i, label in enumerate(label_set):
        rle = rle_set[i]
        label_index = LABELS.index(label)   # Find the index of the label.
        mask = rle2mask(rle, input_shape)
        mask = resize_image(mask, req_shape)
        mask_arr[:,:, label_index] = mask
    
    return mask_arr
        


## Data generator

In [110]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, list_indices, train_df, image_dir, task="train", batch_size=32, image_size=(1400, 2100),
                 train_size=(480, 640), augment=False, n_classes=4, shuffle=True, n_channels=3):
        self._list_indices = list_indices
        self._train_df = train_df
        self._image_dir = image_dir
        self._batch_size = 32
        self._image_size = image_size
        self._train_size = train_size
        self._augment = augment
        self._n_classes = n_classes
        self._shuffle = shuffle
        self._task = task
        self._n_channels = n_channels

        # Reset indices
        self._reset_indices()

    def __len__(self):
        "Return the number of batches per epoch."
        return int(math.floor(len(self._indices)/self._batch_size))

    def __getitem__(self, index):
        "Get a batch of data."
        # generate batch indices. TODO: Try to remove later.
        list_idxs = self._indices[index*self._batch_size:(index+1)*self._batch_size]
        
        print('Batch for index {}'.format(index))

        # Find list IDs
        batch_indices = [self._list_indices[i] for i in list_idxs]

        X, y = self._generate_batch(batch_indices, augment=self._augment, task=self._task)

    def _reset_indices(self):
        self._indices = np.arange(len(self._list_indices))
        if self._shuffle:
            np.random.shuffle(self._indices)
    
    def on_epoch_end(self):
        "What to do after every epoch."
        self._reset_indices()

    def _read_image(self, img_path):
        img = cv2.imread(img_path)
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    def _generate_batch(self, batch_indices, task, augment=False):
        "Generate a batch of data using provided indices."
        out_data_shape = (self._batch_size, *self._train_size, self._n_channels)
        out_label_shape = (self._batch_size, *self._train_size, self._n_classes)
        X = np.empty(out_data_shape)
        Y = np.zeros(out_label_shape)

        # for all batch indices
        for i, id in enumerate(batch_indices):
            # load images
            img = self._load_image(self._image_dir, self._train_df['Image'][id], self._train_size)
            X[i, :, :, :] = img
            
            # load masks
            label_set, rle_set = self._train_df['Label'][id], self._train_df['EncodedPixels'][id]
            mask = rles2arr(label_set, rle_set, self._image_size, self._train_size, self._n_classes)
            Y[i, :, :, :] = mask

        if task is 'train':
            return X, Y
        elif task is 'test':
            return X
        
        raise AttributeError("Task should be either train or test")
        
    def _load_image(self, image_dir, img_name, req_size):
        img_path = os.path.join(image_dir, img_name)
        img = self._read_image(img_path)
        img = resize_image(img, req_size)
        return img



    

## Loss

In [111]:
import tensorflow.keras.backend as K
from tensorflow.keras.losses import binary_crossentropy

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    print(y_true.shape)
    print(y_pred.shape)
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

In [112]:
tf.executing_eagerly()

True

## Training

In [113]:
from sklearn.model_selection import train_test_split
import numpy as np

train_indices, val_indices = train_test_split(train_df.index, test_size=0.2)

train_generator = DataGenerator(
    list_indices=train_indices,
    train_df=train_df,
    image_dir=TRAIN_IMAGE_DIR,
    image_size=INPUT_SHAPE,
    train_size=TRAIN_SIZE,
    task="train",
    augment=True,
    n_classes=4,
    n_channels=3)


val_generator = DataGenerator(
    list_indices=val_indices,
    train_df=train_df,
    image_dir=TRAIN_IMAGE_DIR,
    image_size=INPUT_SHAPE,
    train_size=TRAIN_SIZE,
    task="train",
    augment=True,
    n_classes=4,
    n_channels=3)

In [115]:
import segmentation_models as sm
from tensorflow.keras.optimizers import Adam, Nadam

model = sm.Unet('resnet18', classes=4, input_shape=(*TRAIN_SIZE, 3), activation='sigmoid')
model.compile(optimizer=Nadam(lr=0.0002), loss=bce_dice_loss, metrics=[dice_coef])
model.summary()

(None, None, None, None)
(None, 480, 640, 4)
Model: "model_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               [(None, 480, 640, 3) 0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 480, 640, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_234 (ZeroPadding (None, 486, 646, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 240, 320, 64) 9408        zero_padding2d_234[0][0]         
______________________________________________

In [116]:
model.fit_generator(train_generator, validation_data=val_generator, epochs=N_EPOCHS)

Epoch 1/100
Batch for index 5
Batch for index 14
Batch for index 107
Batch for index 8
Batch for index 97
Batch for index 29
Batch for index 1
Batch for index 38
Batch for index 31
Batch for index 98


KeyboardInterrupt: 

Batch for index 49
Batch for index 66
Batch for index 58
Batch for index 87
Batch for index 28
Batch for index 84
Batch for index 56
Batch for index 94
Batch for index 100
Batch for index 45
Batch for index 110
